In [4]:
from itertools import product, combinations
import keyboard
import pyautogui as g
import time
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR/tesseract'

suits = ('o','e','c','p')
values = (2,3,4,5,6,7,8,9,10,11,12,13,14)
turno = 1
pessoas = 3
pote = 0
pagar = 0
jogar = 0
mao = ['00','00']
mesa = ['00','00','00','00','00']
minhasprobs = [0,0,0,0,0,0,0,0]
adversprobs = [0,0,0,0,0,0,0,0]
bb = 100

def Calcular():
    global turno
    global mao
    global mesa
    global minhasprobs
    global adversprobs
    global jogar
    
    # criar o array de cartas a partir das combinações entre numeros e naipes
    deck = [str(card[0]) + card[1] for card in list(product(values, suits))]
    
    # remover cartas que já estão na mão e/ou na mesa do deck
    deck.pop(deck.index(mao[0]))
    deck.pop(deck.index(mao[1]))
    
    # dependendo do turno, existem mais cartas ou menos cartas já jogadas na mesa para serem removidas
    if turno > 0:
        deck.pop(deck.index(mesa[0]))
        deck.pop(deck.index(mesa[1]))
        deck.pop(deck.index(mesa[2]))
        # numcarta é o número de cartas para serem testadas para completar as 7
        numcarta = 2
        if turno > 1:
            deck.pop(deck.index(mesa[3]))
            numcarta = 1
            if turno > 2:
                deck.pop(deck.index(mesa[4]))
                numcarta = 0
    
    # o loop roda 1 vez para calcular as minhas probabilidades e depois as dos adversários
    # para x == 0 são calculadas as minhas probabilidades
    for x in range(2):
        
        # variáveis que determinam quantas vezes essas jogadas foram obtidas nos testes
        total = 0
        flush = 0
        quad = 0
        full = 0
        trin = 0
        pare = 0
        parr = 0
        nada = 0
        sequencia = 0
        strf = 0
        
        if x == 1:
            # para calcular as probabilidades alheias, existem 2 cartas a
            # mais para serem testadas, já que não sabemos a mão deles
            numcarta = numcarta + 2
            print('adversarios probs:')
        else:
            print('minhas probs:')
        
        # todas as combinações de cartas serão testadas de acordo com o número de cartas que faltam
        for i in list(combinations(deck,numcarta)):
            par = 0
            pares = 0
            trinca = 0
            fullHouse = 0
            quadra = 0
            sequ = 0
            naipeFlush = '0'
            strflush = 0
            total = total + 1
            
            # as cartas usadas na jogada poderam conter cartas da mao, da mesa e as cartas testadas
            # dependendo do turno e de quem são as probabilidades
            if x == 0:
                if numcarta == 2:
                    jogada = [mesa[0],mesa[1],mesa[2], mao[0], mao[1], i[0], i[1]]
                elif numcarta == 1:
                    jogada = [mesa[0],mesa[1],mesa[2], mesa[3], mao[0], mao[1], i[0]]
                elif numcarta == 0:
                    jogada = [mesa[0],mesa[1],mesa[2], mesa[3], mesa[4], mao[0], mao[1]]
            else:  
                if numcarta == 4:
                    jogada = [mesa[0],mesa[1],mesa[2], i[0], i[1], i[2], i[3]]
                elif numcarta == 3:
                    jogada = [mesa[0],mesa[1],mesa[2],mesa[3], i[0], i[1], i[2]]
                elif numcarta == 2:
                    jogada = [mesa[0],mesa[1],mesa[2], mesa[3], mesa[4],i[0], i[1]]
            
            # as cartas testadas serão colocadas em ordem em um novo array de números inteiros
            # se os digitos não forem convertidos em inteiros, o sort() só detecta o primeiro digito para determinar a ordenação
            # isso serve para poder testar sequência ou straight flushs
            seq = [int(jogada[0][:-1]),int(jogada[1][:-1]),int(jogada[2][:-1]),int(jogada[3][:-1]),int(jogada[4][:-1]),int(jogada[5][:-1]),int(jogada[6][:-1])]
            seq.sort()

            # ele contara quantas cartas tem de cada naipe e depois determinará se houve um flush
            countFlush = [0,0,0,0]
            for j in jogada:
                if j[-1] == 'o':
                    countFlush[0] = countFlush[0] + 1
                elif j[-1] == 'e':
                    countFlush[1] = countFlush[1] + 1
                elif j[-1] == 'c':
                    countFlush[2] = countFlush[2] + 1
                elif j[-1] == 'p':
                    countFlush[3] = countFlush[3] + 1
            # armazenará de qual naipe é o flush para calcular depois se houve straight flush
            if countFlush[0] > 4:
                    naipeFlush = 'o'
            elif countFlush[1] > 4:
                naipeFlush = 'e'
            elif countFlush[2] > 4:
                naipeFlush = 'c'
            elif countFlush[3] > 4:
                naipeFlush = 'p'

            # ele verificará se há alguma sequencia, seja começando da 1,2 ou começando a partir da 3 carta
            # para isso, ele primeiro removera as cartas repetidas
            straight = list(set(seq))
            if len(straight) > 4:
                if len(straight) == 5:
                    if straight[0] + 4 == straight[4]:
                        sequ = 1
                elif len(straight) == 6:
                    if straight[0] + 4 == straight[4] or straight[1] + 4 == straight[5]:
                        sequ = 1
                elif len(straight) == 7:
                    if straight[0] + 4 == straight[4] or straight[1] + 4 == straight[5] or straight[2] + 4 == straight[6]:
                        sequ = 1

            # aqui são detectados pares, trincas e quadras
            nums = [0,0,0,0,0,0,0]
            # ele conta quantas copias de carta tem para cada carta
            for p in seq:
                # o array nums conterá as quantidades das cartas
                nums[seq.index(p)] = seq.count(p)
            if 4 in nums:
                quadra = 1
            elif 3 in nums:
                if 2 in nums:
                    fullHouse = 1
                else:
                    trinca = 1
            elif 2 in nums:
                nums.pop(nums.index(2))
                # o código removerá o 2 do array de quatidades
                # se mesmo assim ainda tiver outra dupla, o código detectará pares
                if 2 in nums:
                    pares = 1
                else:
                    par = 1

            
            seqflush = []
            # conferirá se o código anterior indicou sequencia e flush
            if naipeFlush != '0' and sequ == 1:
                # ele vai adicionar ao array cada carta do naipe do flush
                for i in jogada:
                    if naipeFlush == i[-1]:
                        seqflush.append(i)
                # se realmente tiver mais de 4 cartas do mesmo naipe, ele testará se houve sequência
                if len(seqflush) > 4:
                    seqflush2 = []
                    for k in seqflush:
                        # ele adicionará apenas os dígitos a um array de cartas do mesmo naipe
                        seqflush2.append(int(k[:-1]))
                    # vai ordenar essas cartas do mesmo naipe
                    seqflush2.sort()
                    # ele verificará se há alguma sequencia, seja começando da 1,2 ou começando a partir da 3 carta
                    if len(seqflush2) == 5:
                        if seqflush2[0] + 4 == seqflush2[4]:
                            strflush = 1
                    elif len(seqflush2) == 6:
                        if seqflush2[0] + 4 == seqflush2[4] or seqflush2[1] + 4 == seqflush2[5]:
                            strflush = 1
                    elif len(seqflush2) == 7:
                        if seqflush2[0] + 4 == seqflush2[4] or seqflush2[1] + 4 == seqflush2[5] or seqflush2[2] + 4 == seqflush2[6]:
                            strflush = 1


            # ele vai ver qual foi a melhor jogada que o teste encontrou
            if strflush == 1:
                strf = strf + 1

            elif quadra == 1:
                quad = quad + 1

            elif fullHouse == 1:
                full = full + 1

            elif naipeFlush != '0':
                flush = flush + 1

            elif sequ == 1:
                sequencia = sequencia + 1

            elif trinca == 1:
                trin = trin + 1

            elif pares == 1:
                pare = pare + 1

            elif par == 1:
                parr = parr + 1

            else:
                nada = nada + 1
        
        # a partir dos testes, serão lançadas as minhas probabilidades e as dos adversários
        if x == 0:
            print('straight flush :', round((strf/total)*100, 2), '%')
            minhasprobs[7] = round((strf/total)*100)
            print('quadra :', round((quad/total)*100, 2), '%')
            minhasprobs[6] = round((quad/total)*100)
            print('full house :', round((full/total)*100, 2), '%')
            minhasprobs[5] = round((full/total)*100)
            print('flush :', round((flush/total)*100, 2), '%')
            minhasprobs[4] = round((flush/total)*100)
            print('sequencia :', round((sequencia/total)*100, 2), '%')
            minhasprobs[3] = round((sequencia/total)*100)
            print('trinca :', round((trin/total)*100, 2), '%')
            minhasprobs[2] = round((trin/total)*100)
            print('pares :', round((pare/total)*100, 2), '%')
            minhasprobs[1] = round((pare/total)*100)
            print('par :', round((parr/total)*100, 2), '%')
            minhasprobs[0] = round((parr/total)*100)
            print('nada :', round((nada/total)*100, 2), '%')
        else:
            # essas são as probabilidades de pelo menos 1 adversário ter a jogada
            print('straight flush :', round((1 - ((1 - strf/total)**pessoas))*100, 2), '%')
            adversprobs[7] = round((1 - ((1 - strf/total)**pessoas))*100, 2)
            print('quadra :', round((1 - ((1 - quad/total)**pessoas))*100, 2), '%')
            adversprobs[6] = round((1 - ((1 - quad/total)**pessoas))*100, 2)
            print('full house :', round((1 - ((1 - full/total)**pessoas))*100, 2), '%')
            adversprobs[5] = round((1 - ((1 - full/total)**pessoas))*100, 2)
            print('flush :', round((1 - ((1 - flush/total)**pessoas))*100, 2), '%')
            adversprobs[4] = round((1 - ((1 - flush/total)**pessoas))*100, 2)
            print('sequencia :', round((1 - ((1 - sequencia/total)**pessoas))*100, 2), '%')
            adversprobs[3] = round((1 - ((1 - sequencia/total)**pessoas))*100, 2)
            print('trinca :', round((1 - ((1 - trin/total)**pessoas))*100, 2), '%')
            adversprobs[2] = round((1 - ((1 - trin/total)**pessoas))*100, 2)
            print('pares :', round((1 - ((1 - pare/total)**pessoas))*100, 2), '%')
            adversprobs[1] = round((1 - ((1 - pare/total)**pessoas))*100, 2)
            print('par :', round((1 - ((1 - parr/total)**pessoas))*100, 2), '%')
            adversprobs[0] = round((1 - ((1 - parr/total)**pessoas))*100, 2)
            print('nada :', round((1 - ((1 - nada/total)**pessoas))*100, 2), '%')
        passarApostar()

# essa função detecta na tela o valor do pote e o valor que eu preciso pagar pra ir para o próximo round
def scrap():
    global pote
    global pagar
    # captura a regiao onde fica o valor do pote
    img = g.screenshot(region=(890,370, 160, 35))
    img = img.convert('L')
    pote = pytesseract.image_to_string(img)
    # o que foi capturado e não é número vai ser cortado da string
    for i in pote:
        if i.isdigit() == False:
            pote = pote.replace(i,"")
    # se não sobrou nenhum número é por que bugou
    if len(pote) > 0:
        pote = int(pote)
    else:
        print('pote bugou')
        pote = 1000
    print('pote :', pote)

    # captura a região onde fica o valor a ser pago para ir pra proxima rodada
    img = g.screenshot(region=(1220,915, 110, 35))
    img = img.convert('L')
    pagar = pytesseract.image_to_string(img)
    if len(pagar) > 0:
        if pagar[0].isdigit():
            pagar = int(pagar)
        else:
            pagar = 0
    # verifica se o botão realmente está lá pra ser lido, se não, é por deram all win
    elif g.pixelMatchesColor(1220, 900, (120, 22, 9), tolerance = 10):
        pagar = bb*100
    else:
        pagar = 0
    print(pagar)    

def preFlop():
    global bb
    global pessoas
    smallblind = 0
    bigblind = 0
    ppar = 0
    suited = 0
    ssequencia = 0
    valorApostado = 0
    apostar = 0

    time.sleep(2)
    
    # será detectado se na mão existe um par, se são do mesmo naipe, ou se fazem parte de alguma sequencia
    if mao[0][:-1] == mao[1][:-1]:
        ppar = 1
    elif abs(int(mao[0][:-1]) - int(mao[1][:-1])) < 4:
        ssequencia = 4 - abs(int(mao[0][:-1]) - int(mao[1][:-1]))

    if mao[0][-1] == mao[1][-1]:
        suited = 1

    # a partir das variáveis, se estima o valor da mão
    handPower = ppar*55 + suited*14 + ssequencia*5 + (int(mao[0][:-1]) + int(mao[1][:-1]))*3
    print(handPower)
    
    # com base no valor da mão pode-se apostar ou não
    if handPower > 125:
        if pagar >= 10*bb:
            g.moveTo(1265, 915)
            g.click()
        else:
            g.moveTo(1215, 845)
            g.doubleClick()
            g.write(str(10*bb))
            g.moveTo(1465, 915)
            g.click()
    elif handPower > 100:
        if pagar <= 5*bb:
            if pagar == bb:
                g.moveTo(1465, 915)
                g.click()
            else:
                g.moveTo(1265, 915)
                g.click()
        else:
            g.moveTo(1065, 915)
            g.click()
            
    elif handPower > 65:
        if pagar == bb:
            g.moveTo(1265, 915)
            g.click()
        else:
            g.moveTo(1065, 915)
            g.click()
    else:
        g.moveTo(1065, 915)
        g.click()
        
    g.moveTo(1400, 500)    
    Loading()

def passarApostar():
    global minhasprobs
    global adversprobs
    global pote
    global pagar
    global bb
    chances = 0
    
    # calcula aproximadamente a chance de só você ter aquela jogada
    for p in range(8):
        multip = 1
        for q in range(8-p):
            multip = multip*(1 - adversprobs[7-q]/100)
        chances = chances + minhasprobs[p]*multip
        
    # usa várias variáveis para decidir qual o valor das cartas que você tem
    roundPower = chances*(0.8 + (pessoas/5))*(1 + ((pote/bb)/(pagar/bb))/200)
    
    # com base nesse valor, o robo tomará decisões
    if roundPower > 130:
        pay = 200*bb
        bet = 10*bb
    elif roundPower > 100:
        pay = 40*bb
        bet = 5*bb
    elif roundPower > 70:
        pay = 10*bb
        bet = 3*bb
    elif roundPower > 50:
        pay = 6*bb
        bet = 2*bb
    elif roundPower > 40:
        pay = 3*bb
        bet = 1*bb
    elif roundPower > 25:
        pay = 2*bb
        bet = 0
    elif roundPower > 20:
        pay = 1*bb
        bet = 0
    else:
        pay = 0
        bet = 0
        
    bet = bet*turno
    
    if pay/2 >= pote:
        # apostar
        g.moveTo(1215, 845)
        g.doubleClick()
        g.write(str(bet))
        g.moveTo(1465, 915)
        g.click()
    elif pay >= pagar:
        g.moveTo(1265, 915)
        g.click()
    else:
        g.moveTo(1065, 915)
        g.click()
     
    g.moveTo(1400, 500) 
    Loading()
        
def Loading():
    global pessoas
    global turno
    global mao
    global mesa
    global jogar
    jogar = 0
    while jogar == 0:
        # quando um pixel vermelho escuro aparecer na tela, é por que o botão de apostas apareceu
        # isso significa que está na hora do robo tomar alguma decisão
        if g.pixelMatchesColor(1440, 915, (85, 14, 9), tolerance = 10):
            jogar = 1
            
            # ele detectará a cor de 1 pixel para ver se os adversários já desistiram da mão ou estão no jogo ainda
            # com isso, ele saberá contra quantos ele está jogando
            pessoas = 0
            if g.pixelMatchesColor(485, 530, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(510, 270, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(935, 195, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(1340, 270, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1
            if g.pixelMatchesColor(1380, 530, (163, 69, 67), tolerance = 10):
                pessoas = pessoas + 1

            scrap()
            # ele detectara um pixel para saber se a mesa continua verde
            # se não, é porque colocaram 1 carta em cima daquele pixel, indicando que já passou o preflop
            if g.pixelMatchesColor(835, 465, (7, 101, 35), tolerance = 10):
                turno = 0
                # no preflop ele verá as cartas que possui
                # aqui ele testa todas os números para as 2 cartas para saber o valor de cada uma
                for a in range(13):
                    arquivo = 'PokerStarsImages/cartas/' + str(a + 2) + '.jpg'
                    if g.locateOnScreen(arquivo, confidence = 0.75, region = (880,630,60,50), grayscale = True):
                        mao[0] = str(a+2)

                for b in range(13):
                    arquivo = 'PokerStarsImages/cartas/' + str(b + 2) + '.jpg'
                    if g.locateOnScreen(arquivo, confidence = 0.75, region = (960,630,60,50), grayscale = True):
                        mao[1] = str(b+2)

                # aqui ele testa o naipe das 2 cartas
                if g.locateOnScreen('PokerStarsImages/ouros2.png', confidence = 0.9, region = (880,670,60,50), grayscale = True):
                    mao[0] = mao[0] + 'o'  
                elif g.locateOnScreen('PokerStarsImages/copas2.png', confidence = 0.8, region = (880,670,60,50), grayscale = True):
                    mao[0] = mao[0] + 'c'
                elif g.locateOnScreen('PokerStarsImages/espadas2.png', confidence = 0.9, region = (880,670,60,50), grayscale = True):
                    mao[0] = mao[0] + 'e'
                elif g.locateOnScreen('PokerStarsImages/paus2.png', confidence = 0.7, region = (880,670,60,50), grayscale = True):
                    mao[0] = mao[0] + 'p' 

                if g.locateOnScreen('PokerStarsImages/ouros3.png', confidence = 0.85, region = (960,670,60,50), grayscale = True):
                    mao[1] = mao[1] + 'o'  
                elif g.locateOnScreen('PokerStarsImages/copas3.png', confidence = 0.9, region = (960,670,60,50), grayscale = True):
                    mao[1] = mao[1] + 'c'
                elif g.locateOnScreen('PokerStarsImages/espadas3.png', confidence = 0.9, region = (960,670,60,50), grayscale = True):
                    mao[1] = mao[1] + 'e'
                elif g.locateOnScreen('PokerStarsImages/paus3.png', confidence = 0.7, region = (960,670,60,50), grayscale = True):
                    mao[1] = mao[1] + 'p'
                print('mao', mao)
                preFlop()
                
            else:
                # aqui ele detectará as cartas na mesa, vendo seu valor e naipe
                for i in range(5):
                    for j in range(13):
                        arquivo = 'PokerStarsImages/cartas/' + str(j + 2) + '.jpg'
                        if g.locateOnScreen(arquivo, confidence = 0.75, region = (770+i*80,400,40,60), grayscale = True):
                            mesa[i] = str(j+2)
                            break
                    
                    # se alguma carta não for encontrada, o loop se encerrará, pois a próxima carta também não foi jogada ainda
                    if mesa[i] == '00':
                        break

                    if g.locateOnScreen('PokerStarsImages/ouros.jpg', confidence = 0.85, region = (770+i*80,430,40,40)):
                        mesa[i] = mesa[i] + 'o'
                    elif g.locateOnScreen('PokerStarsImages/espadas.jpg', confidence = 0.85, region = (770+i*80,430,40,40)):
                        mesa[i] = mesa[i] + 'e'    
                    elif g.locateOnScreen('PokerStarsImages/copas.jpg', confidence = 0.85, region = (770+i*80,430,40,40)):
                        mesa[i] = mesa[i] + 'c'
                    elif g.locateOnScreen('PokerStarsImages/paus.jpg', confidence = 0.5, region = (770+i*80,430,40,40)):
                        mesa[i] = mesa[i] + 'p'
                print('mesa', mesa)
                
                # com base no número de cartas jogadas, será detectado em qual turno estamos
                if mesa[4] != "00":
                    turno = 3
                elif mesa[3] != "00":
                    turno = 2
                elif mesa[2] != "00":
                    turno = 1
                else:
                    turno = 0
                # após definidas várias variáveis, o código irá calcular as probabilidades
                Calcular()
            
Loading()  

250
100
mao ['7o', '6c']
54
150
100
mao ['10c', '3o']
39
2150
10000
mao ['9p', '13c']
66



KeyboardInterrupt

